In [1]:
# !pip install pingouin
# !pip install regex
# !pip install altair
import numpy as np
import pandas as pd
from scipy import stats
# import pingouin as pg
import regex as re
import altair as alt
from altair import *
import requests

pd.set_option("display.max_columns", 999)

In [2]:
# !pip install nba_api
# !pip install sportsreference
from nba_api.stats.static import players
from nba_api.stats.static import teams 
from nba_api.stats.endpoints import playergamelog
from nba_api.stats.library.parameters import SeasonAll
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.endpoints import shotchartdetail
from nba_api.stats.endpoints import playerdashptshotdefend
from nba_api.stats.endpoints import draftcombineplayeranthro
from nba_api.stats.library.parameters import *
from nba_api.stats.static import players
import json
import seaborn as sns
import matplotlib
from matplotlib import pyplot as plt
from sportsreference.ncaab.boxscore import Boxscore
from sportsreference.ncaab.player import AbstractPlayer
import pprint
from sportsreference.ncaab.teams import Teams
from sportsreference.ncaab.roster import Roster
from sportsreference.ncaab.roster import Player
from sportsreference.nba.teams import Teams as NBA_TEAMS
from sportsreference.nba.roster import Roster as NBA_ROSTER
from sportsreference.nba.roster import Player as NBA_PLAYER
import random

In [3]:
player_dict = players.get_players()
team_dict = teams.get_teams()

individualPlayer_dict = [player for player in player_dict if player['full_name'] == 'Trae Young'][0]
individualPlayer_id = individualPlayer_dict['id']

individualTeam_dict = [x for x in team_dict if x['full_name'] == 'Atlanta Hawks'][0]
individualTeam_id = individualTeam_dict['id']

In [4]:
years = ['2013','2014','2015','2016','2017','2018']
wingspanDfs = []
for year in years:
    wingspanDf = pd.DataFrame(draftcombineplayeranthro.DraftCombinePlayerAnthro(league_id='00',season_year=year).get_data_frames()[0])
    wingspanDfs.append(wingspanDf)

wingspanDfFull = pd.concat(wingspanDfs)
wingspanDfFull

,TEMP_PLAYER_ID,PLAYER_ID,FIRST_NAME,LAST_NAME,PLAYER_NAME,POSITION,HEIGHT_WO_SHOES,HEIGHT_WO_SHOES_FT_IN,HEIGHT_W_SHOES,HEIGHT_W_SHOES_FT_IN,WEIGHT,WINGSPAN,WINGSPAN_FT_IN,STANDING_REACH,STANDING_REACH_FT_IN,BODY_FAT_PCT,HAND_LENGTH,HAND_WIDTH
0,None,203500,Steven,Adams,Steven Adams,C,82.75,6' 10.75'',84.00,7' 0'',254.5,88.50,7' 4.5'',109.5,9' 1.5'',6.65,9.50,11.00
1,None,203505,Vander,Blue,Vander Blue,SG,75.50,6' 3.5'',77.25,6' 5.25'',197.4,78.00,6' 6'',100.5,8' 4.5'',4.60,8.50,9.50
2,None,203485,Lorenzo,Brown,Lorenzo Brown,PG,76.00,6' 4'',77.25,6' 5.25'',189.2,79.00,6' 7'',100.0,8' 4'',3.95,8.25,8.50
3,None,203493,Reggie,Bullock,Reggie Bullock,SG-SF,77.75,6' 5.75'',79.00,6' 7'',199.8,80.75,6' 8.75'',102.0,8' 6'',6.95,8.50,9.00
4,None,203504,Trey,Burke,Trey Burke,PG,71.75,5' 11.75'',73.25,6' 1.25'',187,77.50,6' 5.5'',97.5,8' 1.5'',6.90,8.25,8.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,1629023,1629023,PJ,Washington,PJ Washington,PF,78.50,6' 6.5'',80.00,6' 8'',223,86.50,7' 2.5'',107.0,8' 11'',6.85,9.00,10.25
65,1629024,1629024,Austin,Wiley,Austin Wiley,C,80.25,6' 8.25'',82.00,6' 10'',249.4,89.00,7' 5'',111.0,9' 3'',5.00,9.75,10.25
66,1629025,1629025,Kris,Wilkes,Kris Wilkes,SF,78.50,6' 6.5'',79.50,6' 7.5'',205,82.25,6' 10.25'',103.5,8' 7.5'',5.15,8.75,9.25
67,1629026,1629026,Kenrich,Williams,Kenrich Williams,SF,78.00,6' 6'',79.50,6' 7.5'',200.4,79.25,6' 7.25'',103.5,8' 7.5'',5.95,8.50,9.75


In [8]:
seasons = ['2014-15','2015-16','2016-17','2017-18','2018-19']
shotchartDfs = []
for season in seasons:   
    shotchartDf = pd.DataFrame(shotchartdetail.ShotChartDetail(team_id = '0', player_id= '0', season_nullable = season).get_data_frames()[0])
    shotchartDfs.append(shotchartDf)
shotchartDfFull = pd.concat(shotchartDfs)

ReadTimeout: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)

In [ ]:
shotdefenseDfs = []
for season in seasons:
    shotdefenseDf = pd.DataFrame(playerdashptshotdefend.PlayerDashPtShotDefend(team_id = '0',player_id='0',season = season).get_data_frames()[0]).rename(columns={'CLOSE_DEF_playerId':'CLOSE_DEF_PERSON_ID'})
    shotdefenseDfs.append(shotdefenseDf)

shotdefenseDfFull = pd.concat(shotdefenseDfs)

In [ ]:
shotDefenseAndWingspan = pd.merge(left=shotdefenseDfFull,right=wingspanDfFull,how='left',left_on='CLOSE_DEF_PERSON_ID',right_on='PLAYER_ID')
shotDefenseAndWingspan = shotDefenseAndWingspan.dropna(subset=['STANDING_REACH','PCT_PLUSMINUS'])
shotDefenseAndWingspan.head()

,CLOSE_DEF_PERSON_ID,GP,G,DEFENSE_CATEGORY,FREQ,D_FGM,D_FGA,D_FG_PCT,NORMAL_FG_PCT,PCT_PLUSMINUS,TEMP_PLAYER_ID,PLAYER_ID,FIRST_NAME,LAST_NAME,PLAYER_NAME,POSITION,HEIGHT_WO_SHOES,HEIGHT_WO_SHOES_FT_IN,HEIGHT_W_SHOES,HEIGHT_W_SHOES_FT_IN,WEIGHT,WINGSPAN,WINGSPAN_FT_IN,STANDING_REACH,STANDING_REACH_FT_IN,BODY_FAT_PCT,HAND_LENGTH,HAND_WIDTH
5,203897,74,74,Overall,1.000,283.0,568.0,0.498,0.433,0.065,14033,203897.0,Zach,LaVine,Zach LaVine,PG,76.5,6' 4.5'',77.75,6' 5.75'',180.9,80.25,6' 8.25'',100.0,8' 4'',4.70,8.25,8.75
6,203515,43,43,Overall,1.000,95.0,208.0,0.457,0.421,0.035,None,203515.0,Phil,Pressey,Phil Pressey,PG,69.5,5' 9.5'',71.50,5' 11.5'',177,74.25,6' 2.25'',92.0,7' 8'',4.70,7.75,9.75
18,203473,54,54,Overall,1.000,135.0,327.0,0.413,0.468,-0.055,None,203473.0,Dewayne,Dedmon,Dewayne Dedmon,C,82.0,6' 10'',83.50,6' 11.5'',238.8,88.00,7' 4'',109.0,9' 1'',8.35,9.00,11.00
35,203921,34,34,Overall,1.000,91.0,188.0,0.484,0.441,0.043,14016,203921.0,Cleanthony,Early,Cleanthony Early,SF,78.0,6' 6'',79.25,6' 7.25'',209.4,82.75,6' 10.75'',101.5,8' 5.5'',4.45,8.50,9.00
48,203897,74,60,3 Pointers,0.343,77.0,195.0,0.395,0.349,0.046,14033,203897.0,Zach,LaVine,Zach LaVine,PG,76.5,6' 4.5'',77.75,6' 5.75'',180.9,80.25,6' 8.25'',100.0,8' 4'',4.70,8.25,8.75


In [ ]:
shotDefenseAndWingspan['POSITION'].unique()

In [72]:
Base = alt.Chart(shotDefenseAndWingspan).mark_circle(size=30).encode(
x=alt.X(
  'STANDING_REACH',
  axis=alt.Axis(),
  scale=Scale(domain=[85, 120]),
  title=None
  ),
  y=alt.Y(
  'PCT_PLUSMINUS',
  axis=alt.Axis(),
  scale=Scale(domain=[-0.7, 0.7]),
  title='Standing Reach X Effect on FG Percentage'
  ),
  color = alt.Color('POSITION',scale = alt.Scale(scheme='bluegreen'))
 ).properties(
   width=900,
   height=300
 )

Base + Base.transform_regression('STANDING_REACH','PCT_PLUSMINUS').mark_line(color='black')

alt.LayerChart(...)

In [74]:
# As expected, the relationship between both variables is negative(lower plus minus==better defense), 
# Below we test the significance of the relationship

correlation, p_value = stats.pearsonr(shotDefenseAndWingspan['STANDING_REACH'], shotDefenseAndWingspan['PCT_PLUSMINUS'])
print('Correlation coefficient:', correlation)
print('P-value:', p_value)

alpha = 0.05  # significance level

if p_value < alpha:
    print('There is a statistically significant correlation between x and y.')
else:
    print('There is no statistically significant correlation between x and y.')


Correlation coefficient: -0.12004915766518366
P-value: 2.297513643483127e-15
There is a statistically significant correlation between x and y.
